In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [ ]:
def build_model():
    
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=(2,)),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

model = build_model()


In [ ]:
model.summary()

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    print('.', end='')
    if epoch % 100 == 99: 
        print("val_loss = %.3f" % logs["val_loss"])

EPOCHS = 1000


In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()



Fix overtraining by adding an early stopping.

In [ ]:
def simple_generator(batch_size = 128):
    while True:
        inputs  = np.random.random((batch_size, 2))
        targets = np.product(inputs, axis = 1)
        yield (inputs, targets)

In [ ]:
next(simple_generator(batch_size = 5))

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
#early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#history = model.fit_generator(simple_generator(), verbose=0, callbacks=[early_stop, PrintDot()],
#                             steps_per_epoch=200, epochs=1)
history = model.fit_generator(simple_generator(), steps_per_epoch=200, epochs=5)

#plot_history(history)


Plot prediction vs truth:

In [ ]:
inputs, test_labels = next(simple_generator())
test_predictions = model.predict(inputs)

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
